<a href="https://colab.research.google.com/github/SHUDengwei/Pkmer-Docs/blob/main/01_%E5%81%A5%E5%BA%B7%E6%95%B0%E6%8D%AE%E8%A7%A3%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 模块1：健康数据智能解析

## 🎯 学习目标

在这个模块中，你将学会：

1. **使用 Pydantic 定义数据模型** - 类型安全的数据验证
2. **JSON Schema 生成** - 从 Pydantic 模型自动生成 JSON Schema
3. **Gemini 结构化输出** - 使用 `response_schema` 强制 JSON 格式输出
4. **自然语言 → 结构化数据** - 将口语化健康记录转换为标准格式

## 🔑 核心技术

- **Pydantic v2** - 数据验证与序列化
- **Gemini JSON Mode** - `response_mime_type = "application/json"`
- **Schema Validation** - 确保 AI 输出符合预期结构

## 步骤1：安装依赖

In [ ]:
# 安装必要的包
%pip install -q google-genai pydantic

print("✅ 依赖安装完成！")

✅ 依赖安装完成！


## 步骤2：配置 Gemini API

In [ ]:
import os
from google import genai
from google.genai import types

# 如果在 Colab 中运行
try:
    from google.colab import userdata
    api_key = userdata.get('GEMINI_API_KEY')
except ImportError:
    # 如果在本地运行，从环境变量读取
    api_key = os.getenv('GEMINI_API_KEY')

if not api_key:
    raise ValueError("❌ 请先配置 GEMINI_API_KEY")

client = genai.Client(api_key=api_key)
print("✅ Gemini API 配置成功！")

✅ Gemini API 配置成功！


## 步骤3：使用 Pydantic 定义健康数据模型

我们将创建一套完整的数据模型来描述健康记录的结构。

### 数据模型层次结构：
```
DailyHealthData (每日健康数据)
├── Sleep (睡眠数据)
├── Meal (餐食数据) × N
├── Exercise (运动数据)
└── Mood (情绪/压力数据)
```


In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

# 1. 睡眠数据模型
class Sleep(BaseModel):
    """睡眠记录"""
    bedtime: str = Field(description="上床时间，格式：HH:MM")
    wake_time: str = Field(description="起床时间，格式：HH:MM")
    duration_hours: float = Field(description="睡眠时长（小时）", ge=0, le=24)
    quality: Literal["很好", "良好", "一般", "较差", "很差"] = Field(description="睡眠质量评价")
    interruptions: int = Field(default=0, description="夜间醒来次数", ge=0)
    notes: Optional[str] = Field(default=None, description="额外备注")

# 2. 餐食数据模型
class Meal(BaseModel):
    """单次餐食记录"""
    meal_type: Literal["breakfast", "lunch", "dinner", "snack"] = Field(description="餐次类型")
    time: str = Field(description="用餐时间，格式：HH:MM")
    foods: List[str] = Field(description="食物列表")
    notes: Optional[str] = Field(default=None, description="备注（如：油腻、健康等）")

# 3. 运动活动模型
class Activity(BaseModel):
    """单次运动活动"""
    type: str = Field(description="运动类型，如：跑步、走路、游泳")
    duration_minutes: int = Field(description="运动时长（分钟）", ge=0)
    intensity: Literal["低", "中", "高"] = Field(description="运动强度")

# 4. 运动数据模型
class Exercise(BaseModel):
    """每日运动记录"""
    activities: List[Activity] = Field(default_factory=list, description="运动活动列表")
    total_duration_minutes: int = Field(description="总运动时长（分钟）", ge=0)
    steps: Optional[int] = Field(default=None, description="步数", ge=0)
    notes: Optional[str] = Field(default=None, description="额外备注")

# 5. 情绪/压力数据模型
class Mood(BaseModel):
    """情绪与压力记录"""
    stress_level: Literal["低", "中", "高", "极高"] = Field(description="压力水平")
    mood_score: int = Field(description="心情评分（0-100）", ge=0, le=100)
    notes: Optional[str] = Field(default=None, description="情绪相关备注")

# 6. 每日健康数据（汇总）
class DailyHealthData(BaseModel):
    """完整的每日健康记录"""
    date: str = Field(description="日期，格式：YYYY-MM-DD")
    sleep: Sleep = Field(description="睡眠数据")
    meals: List[Meal] = Field(description="餐食记录列表")
    exercise: Exercise = Field(description="运动数据")
    mood: Mood = Field(description="情绪与压力数据")

print("✅ Pydantic 模型定义完成！")

✅ Pydantic 模型定义完成！


### 🔍 查看生成的 JSON Schema

Pydantic 可以自动生成符合 JSON Schema 规范的 schema，这正是 Gemini API 需要的格式！


In [ ]:
import json

# 查看 DailyHealthData 的 JSON Schema
schema = DailyHealthData.model_json_schema()
print("📋 DailyHealthData 的 JSON Schema：\n")
print(json.dumps(schema, indent=2, ensure_ascii=False))

📋 DailyHealthData 的 JSON Schema：

{
  "$defs": {
    "Activity": {
      "description": "单次运动活动",
      "properties": {
        "type": {
          "description": "运动类型，如：跑步、走路、游泳",
          "title": "Type",
          "type": "string"
        },
        "duration_minutes": {
          "description": "运动时长（分钟）",
          "minimum": 0,
          "title": "Duration Minutes",
          "type": "integer"
        },
        "intensity": {
          "description": "运动强度",
          "enum": [
            "低",
            "中",
            "高"
          ],
          "title": "Intensity",
          "type": "string"
        }
      },
      "required": [
        "type",
        "duration_minutes",
        "intensity"
      ],
      "title": "Activity",
      "type": "object"
    },
    "Exercise": {
      "description": "每日运动记录",
      "properties": {
        "activities": {
          "description": "运动活动列表",
          "items": {
            "$ref": "#/$defs/Activity"
          },
          "tit

## 步骤4：准备示例数据（小明的健康记录）

这是我们的测试用户"小明"的自然语言健康记录。注意：**这是真实用户会说的话，不是结构化数据**。


In [ ]:
# 小明的自然语言健康记录
natural_language_input = """
周一（2025-11-11）的健康情况：
昨晚大概11点半才睡，早上7点闹钟响了才起来，感觉有点困。
早餐在楼下买的煎饼果子和豆浆，挺好吃的。
中午公司食堂吃的盖浇饭，有点油腻，还喝了一杯奶茶。
下午一直坐着开会，基本没怎么动。
晚上加班到9点，叫了外卖麻辣烫，回家后刷手机刷到凌晨1点多才睡。
整体感觉工作压力有点大，心情一般，晚上刷手机到很晚才睡。
"""

print("📝 小明的原始健康记录（自然语言）：")
print(natural_language_input)

📝 小明的原始健康记录（自然语言）：

周一（2025-11-11）的健康情况：
昨晚大概11点半才睡，早上7点闹钟响了才起来，感觉有点困。
早餐在楼下买的煎饼果子和豆浆，挺好吃的。
中午公司食堂吃的盖浇饭，有点油腻，还喝了一杯奶茶。
下午一直坐着开会，基本没怎么动。
晚上加班到9点，叫了外卖麻辣烫，回家后刷手机刷到凌晨1点多才睡。
整体感觉工作压力有点大，心情一般，晚上刷手机到很晚才睡。



## 步骤5：配置 Gemini 使用 JSON Schema 输出

这是本模块的**核心技术**：

1. 设置 `response_mime_type = "application/json"` - 强制 JSON 输出
2. 设置 `response_schema` - 使用 Pydantic 生成的 Schema
3. Gemini 会严格按照 schema 格式输出


In [ ]:
# 创建 JSON Schema 配置
json_config = types.GenerateContentConfig(
    response_mime_type="application/json",
    response_schema=DailyHealthData,  # 直接传入 Pydantic 模型
)

print("✅ Gemini 已配置为使用 DailyHealthData 的 JSON Schema 输出")

✅ Gemini 已配置为使用 DailyHealthData 的 JSON Schema 输出


## 步骤6：让 AI 解析自然语言健康记录

现在我们让 Gemini 将自然语言转换为结构化的 JSON 数据。

In [ ]:
# 构建 prompt
prompt = f"""
请将以下自然语言健康记录解析为结构化的JSON数据。

健康记录：
{natural_language_input}

要求：
1. 提取睡眠时间、餐食、运动、情绪等信息
2. 推断合理的数值（如睡眠质量评分、压力水平）
3. 如果某些信息缺失，请根据上下文合理推断
4. 严格遵循提供的 JSON Schema
"""

print("🤖 正在调用 Gemini 进行解析...\n")

# 调用 API
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt,
    config=json_config
)

# 获取 JSON 响应
json_output = response.text
print("✅ AI 解析完成！\n")
print("📊 结构化输出（JSON）：")
print(json_output)

🤖 正在调用 Gemini 进行解析...

✅ AI 解析完成！

📊 结构化输出（JSON）：
{"date":"2025-11-11","sleep":{"bedtime":"23:30","wake_time":"07:00","duration_hours":7.5,"quality":"较差","interruptions":0,"notes":"感觉有点困"},"meals":[{"meal_type":"breakfast","time":"07:30","foods":["煎饼果子","豆浆"],"notes":"挺好吃的"},{"meal_type":"lunch","time":"12:30","foods":["盖浇饭","奶茶"],"notes":"有点油腻"},{"meal_type":"dinner","time":"21:00","foods":["麻辣烫"],"notes":null}],"exercise":{"activities":[],"total_duration_minutes":0,"steps":null,"notes":"下午一直坐着开会，基本没怎么动"},"mood":{"stress_level":"高","mood_score":55,"notes":"工作压力有点大，心情一般，晚上刷手机到很晚才睡"}}


## 步骤7：使用 Pydantic 验证和解析输出

Pydantic 的强大之处在于：不仅生成 Schema，还能验证数据！

In [ ]:
import json

# 将 JSON 字符串解析为 Python dict
data_dict = json.loads(json_output)

# 使用 Pydantic 验证和解析
try:
    health_data = DailyHealthData.model_validate(data_dict)
    print("✅ 数据验证成功！\n")

    # 现在我们可以安全地访问数据（有类型提示）
    print("=" * 60)
    print(f"📅 日期: {health_data.date}")
    print(f"\n😴 睡眠信息:")
    print(f"  - 上床时间: {health_data.sleep.bedtime}")
    print(f"  - 起床时间: {health_data.sleep.wake_time}")
    print(f"  - 睡眠时长: {health_data.sleep.duration_hours}小时")
    print(f"  - 睡眠质量: {health_data.sleep.quality}")

    print(f"\n🍽️ 餐食记录:")
    for meal in health_data.meals:
        print(f"  - {meal.meal_type} ({meal.time}): {', '.join(meal.foods)}")
        if meal.notes:
            print(f"    备注: {meal.notes}")

    print(f"\n🏃 运动数据:")
    if health_data.exercise.activities:
        for activity in health_data.exercise.activities:
            print(f"  - {activity.type}: {activity.duration_minutes}分钟（强度：{activity.intensity}）")
    else:
        print(f"  - 总时长: {health_data.exercise.total_duration_minutes}分钟")
        print(f"  - 备注: {health_data.exercise.notes}")

    print(f"\n😊 情绪与压力:")
    print(f"  - 压力水平: {health_data.mood.stress_level}")
    print(f"  - 心情评分: {health_data.mood.mood_score}/100")
    print(f"  - 备注: {health_data.mood.notes}")
    print("=" * 60)

except Exception as e:
    print(f"❌ 数据验证失败: {e}")

✅ 数据验证成功！

📅 日期: 2025-11-11

😴 睡眠信息:
  - 上床时间: 23:30
  - 起床时间: 07:00
  - 睡眠时长: 7.5小时
  - 睡眠质量: 较差

🍽️ 餐食记录:
  - breakfast (07:30): 煎饼果子, 豆浆
    备注: 挺好吃的
  - lunch (12:30): 盖浇饭, 奶茶
    备注: 有点油腻
  - dinner (21:00): 麻辣烫

🏃 运动数据:
  - 总时长: 0分钟
  - 备注: 下午一直坐着开会，基本没怎么动

😊 情绪与压力:
  - 压力水平: 高
  - 心情评分: 55/100
  - 备注: 工作压力有点大，心情一般，晚上刷手机到很晚才睡


## 步骤8：对比实验 - 有 Schema vs 无 Schema

让我们看看**不使用 Schema** 时，AI 的输出会怎样。

In [ ]:
prompt_no_schema = f"""
请将以下健康记录转换为JSON格式：

{natural_language_input}
"""

print("🔍 实验：不使用 Schema 的输出\n")
print("=" * 60)

# 调用 API（不使用 Schema）
response_no_schema = client.models.generate_content(
    model='gemini-2.0-flash-exp',
    contents=prompt_no_schema
)
print(response_no_schema.text)
print("=" * 60)

print("\n⚠️ 问题：")
print("1. 可能包含 Markdown 代码块标记 ```json")
print("2. 字段名称可能不一致（如：sleepTime vs bedtime）")
print("3. 数据类型可能不符合预期（如：字符串 vs 数字）")
print("4. 可能缺少必需字段")
print("5. 枚举值可能不符合预期（如：'good' vs '良好'）")
print("\n✅ 使用 Pydantic + Schema 可以完全避免这些问题！")

🔍 实验：不使用 Schema 的输出

```json
{
  "date": "2025-11-11",
  "dayOfWeek": "周一",
  "sleep": {
    "bedtime": "23:30",
    "wakeUpTime": "07:00",
    "sleepQuality": "有点困",
    "notes": "晚上刷手机到很晚才睡"
  },
  "meals": [
    {
      "mealTime": "早餐",
      "description": "楼下买的煎饼果子和豆浆，挺好吃的"
    },
    {
      "mealTime": "午餐",
      "description": "公司食堂吃的盖浇饭，有点油腻，还喝了一杯奶茶"
    },
    {
      "mealTime": "晚餐",
      "description": "外卖麻辣烫"
    }
  ],
  "activity": {
    "afternoon": "一直坐着开会，基本没怎么动"
  },
  "work": {
    "notes": "加班到9点, 工作压力有点大"
  },
  "mood": "心情一般",
  "overallFeeling": "整体感觉工作压力有点大，心情一般，晚上刷手机到很晚才睡"
}
```

⚠️ 问题：
1. 可能包含 Markdown 代码块标记 ```json
2. 字段名称可能不一致（如：sleepTime vs bedtime）
3. 数据类型可能不符合预期（如：字符串 vs 数字）
4. 可能缺少必需字段
5. 枚举值可能不符合预期（如：'good' vs '良好'）

✅ 使用 Pydantic + Schema 可以完全避免这些问题！


## 步骤9：解析更多天的数据

让我们解析小明周二的健康记录。

In [ ]:
# 周二的健康记录
tuesday_input = """
周二（2025-11-12）的情况：
昨晚12点睡的，早上6点半起来，睡了6个半小时，还挺累的。
早上没吃早饭，直接去公司了。
午饭吃的轻食沙拉配鸡胸肉，感觉还挺健康的。
下午心情不错，工作也比较顺利。
晚饭吃了糙米饭和清蒸鱼，还散步了30分钟。
"""

# 解析周二数据
print("🤖 正在解析周二的数据...\n")

prompt_tuesday = f"""
请将以下自然语言健康记录解析为结构化的JSON数据：

{tuesday_input}

要求：
1. 严格遵循提供的 JSON Schema
2. 如果某些信息缺失（如没吃早餐），请在meals列表中排除该餐次
3. 根据描述推断合理的评分和评级
"""

response_tuesday = client.models.generate_content(
    model='gemini-2.0-flash-exp',
    contents=prompt_tuesday,
    config=json_config
)
tuesday_data = DailyHealthData.model_validate_json(response_tuesday.text)

print("✅ 周二数据解析成功！\n")
print("=" * 60)
print(f"📅 日期: {tuesday_data.date}")
print(f"😴 睡眠: {tuesday_data.sleep.duration_hours}小时 - {tuesday_data.sleep.quality}")
print(f"🍽️ 用餐: {len(tuesday_data.meals)}次")
for meal in tuesday_data.meals:
    print(f"  - {meal.meal_type}: {', '.join(meal.foods)}")
print(f"🏃 运动: {tuesday_data.exercise.total_duration_minutes}分钟")
print(f"😊 心情: {tuesday_data.mood.mood_score}/100 (压力: {tuesday_data.mood.stress_level})")
print("=" * 60)

🤖 正在解析周二的数据...

✅ 周二数据解析成功！

📅 日期: 2025-11-12
😴 睡眠: 6.5小时 - 一般
🍽️ 用餐: 2次
  - lunch: 轻食沙拉, 鸡胸肉
  - dinner: 糙米饭, 清蒸鱼
🏃 运动: 30分钟
😊 心情: 75/100 (压力: 低)
